<h1>logistic regression without applying polynomin</h1>

do regularization with modal 'liblinear'

# <h4>accuracy: 0.99</h4>

In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [2]:
dataPath = '../../preprocess_data_set/w2v_vec250.csv'


df_train = pd.read_csv(dataPath)
df_train.shape

(44898, 251)

In [3]:
# randomly shuffle the rows
df_train = df_train.sample(frac=1, random_state=42)

print('Shuffled dataframe:')
df_train

Shuffled dataframe:


,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
22216,16.491493,95.172600,56.424232,-68.599670,-60.912342,7.188227,-13.449571,97.102104,-82.553505,-21.996725,...,-70.748657,19.197559,107.534119,-35.678185,11.226527,82.967758,-24.779600,17.169485,5.379145,1
27917,-72.118202,27.964672,-7.027583,10.027310,-131.827057,-7.486910,-44.090725,133.783813,-194.863373,17.914831,...,-60.451050,107.544975,108.068588,-26.898706,78.787643,99.118088,-22.362482,73.297134,-33.535873,1
25007,3.756721,-40.062057,68.558792,-42.674545,-109.676819,-14.668758,-28.305134,51.749004,-50.748463,-13.276784,...,-41.899300,57.503399,69.499634,-32.050545,-4.113337,68.741402,-36.689514,62.024269,-38.039894,1
1377,-5.599266,18.342928,7.739429,-44.109142,-22.294226,20.409508,-6.085033,5.827918,1.943156,-9.718429,...,5.241331,14.465764,17.903656,-52.080715,27.857340,58.687279,-32.958290,16.111702,3.400345,0
32476,-15.102126,-2.364689,-13.435221,-36.039989,-36.147312,-1.247078,-7.055103,24.922768,-67.551460,-39.405941,...,10.585206,34.318207,20.174820,-13.226823,-8.740134,27.602730,3.126579,42.755436,-10.994482,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,60.789886,29.569677,43.278191,-118.951492,93.608200,48.588161,14.821746,34.658852,-32.712147,36.027985,...,-86.828911,18.453732,34.973248,-5.923214,41.222439,73.040588,-30.472477,45.009151,-43.001385,0
44732,-36.097504,-40.508411,1.085567,-5.865716,-61.894810,6.040255,30.072403,59.028919,-48.924541,-14.242128,...,-12.034688,33.752903,11.723240,-10.884808,-36.227303,1.101705,-45.026833,18.150423,-5.883533,1
38158,59.211033,2.794413,25.503954,-22.656900,-72.267227,-49.418575,-19.126442,35.768223,-35.190441,8.482786,...,-14.868692,-22.925610,-12.410628,-65.246872,-4.386832,60.229225,-41.607170,68.239143,-20.501524,1
860,71.339561,16.005495,51.056477,-179.901749,-14.259531,126.382141,-4.463017,42.901329,53.434349,-14.907699,...,-106.613243,-39.113091,48.542309,-67.660194,58.561333,206.557541,-90.538963,154.817764,-92.831696,0


In [4]:
df_train = df_train.reset_index()

df_train = df_train.drop('index', axis=1)
df_train

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
0,16.491493,95.172600,56.424232,-68.599670,-60.912342,7.188227,-13.449571,97.102104,-82.553505,-21.996725,...,-70.748657,19.197559,107.534119,-35.678185,11.226527,82.967758,-24.779600,17.169485,5.379145,1
1,-72.118202,27.964672,-7.027583,10.027310,-131.827057,-7.486910,-44.090725,133.783813,-194.863373,17.914831,...,-60.451050,107.544975,108.068588,-26.898706,78.787643,99.118088,-22.362482,73.297134,-33.535873,1
2,3.756721,-40.062057,68.558792,-42.674545,-109.676819,-14.668758,-28.305134,51.749004,-50.748463,-13.276784,...,-41.899300,57.503399,69.499634,-32.050545,-4.113337,68.741402,-36.689514,62.024269,-38.039894,1
3,-5.599266,18.342928,7.739429,-44.109142,-22.294226,20.409508,-6.085033,5.827918,1.943156,-9.718429,...,5.241331,14.465764,17.903656,-52.080715,27.857340,58.687279,-32.958290,16.111702,3.400345,0
4,-15.102126,-2.364689,-13.435221,-36.039989,-36.147312,-1.247078,-7.055103,24.922768,-67.551460,-39.405941,...,10.585206,34.318207,20.174820,-13.226823,-8.740134,27.602730,3.126579,42.755436,-10.994482,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,60.789886,29.569677,43.278191,-118.951492,93.608200,48.588161,14.821746,34.658852,-32.712147,36.027985,...,-86.828911,18.453732,34.973248,-5.923214,41.222439,73.040588,-30.472477,45.009151,-43.001385,0
44894,-36.097504,-40.508411,1.085567,-5.865716,-61.894810,6.040255,30.072403,59.028919,-48.924541,-14.242128,...,-12.034688,33.752903,11.723240,-10.884808,-36.227303,1.101705,-45.026833,18.150423,-5.883533,1
44895,59.211033,2.794413,25.503954,-22.656900,-72.267227,-49.418575,-19.126442,35.768223,-35.190441,8.482786,...,-14.868692,-22.925610,-12.410628,-65.246872,-4.386832,60.229225,-41.607170,68.239143,-20.501524,1
44896,71.339561,16.005495,51.056477,-179.901749,-14.259531,126.382141,-4.463017,42.901329,53.434349,-14.907699,...,-106.613243,-39.113091,48.542309,-67.660194,58.561333,206.557541,-90.538963,154.817764,-92.831696,0


In [5]:
train_y = df_train['target']
train_x = df_train.drop(columns=['target'])

In [6]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3,random_state=109) # 70% training and 30% test

robust_scaler = RobustScaler().fit(train_x.values)
trans_x = robust_scaler.transform(train_x.values)
# trans_test = robust_scaler.transform(test.values)

In [8]:
param_grid = {"C": [.01, .1, 1], "penalty": ('l1','l2')}
logisticClassifier = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter =300, penalty = 'l1')
grid_train = GridSearchCV(logisticClassifier, param_grid, cv = 5)
grid_train.fit(trans_x, train_y)

print('best_score: ', grid_train.best_score_)

best_score:  0.9906232012285038


In [9]:
pca = PCA(n_components = 200)
pca.fit(train_x.values)
vals_2d = pca.transform(train_x.values)

colors = ['red' if t == 1 else 'blue' for t in train_y]
for i in range(len(vals_2d)):
    plt.scatter(x = vals_2d[i,0], y = vals_2d[i,1], color = colors[i])

In [ ]:
# Create a pipeline with PCA and logistic regression
pc_num = 200

pipe = Pipeline([
    ('pca', PCA(n_components = pc_num)), # choose number of components to keep
    ('clf', LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter = 1000, penalty = 'l1'))
])

pipe.fit(trans_x, train_y)

# Create a scree plot
plt.plot(np.arange(1, pipe['pca'].n_components_+1), pipe['pca'].explained_variance_ratio_, 'bo-')
plt.xlabel('Number of PCs')
plt.ylabel('Proportion of Variance Explained')
plt.title('Scree Plot')
plt.show()

<h3>Real prediction on testing data</h3>

In [7]:
robust_scaler = RobustScaler().fit(X_train.values)
trans_x = robust_scaler.transform(X_train.values)
trans_test = robust_scaler.transform(X_test.values)

In [8]:
param_grid = {"C": [.01, .1], "penalty": ('l1','l2')}
logisticClassifier = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter = 300, penalty = 'l1')

# find best parameter with cross validation
grid_test = GridSearchCV(logisticClassifier, param_grid, cv=5)
grid_test.fit(trans_x, y_train)


# predictions = grid_test.predict_proba(trans_test)[:,1]

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced', max_iter=300,
                                          penalty='l1', solver='liblinear'),
             param_grid={'C': [0.01, 0.1], 'penalty': ('l1', 'l2')})

In [9]:
print("Average performance of the model: {:.3f}".format(grid_test.best_score_))

Average performance of the model: 0.988


In [10]:
grid_test.cv_results_

{'mean_fit_time': array([0.64377737, 1.63335056, 1.9388998 , 2.94189048]),
 'std_fit_time': array([0.0138541 , 0.02985221, 0.42730141, 0.7152484 ]),
 'mean_score_time': array([0.00484157, 0.00335221, 0.0029634 , 0.00396876]),
 'std_score_time': array([1.95710589e-03, 4.85861766e-04, 1.45404844e-05, 1.84699166e-05]),
 'param_C': masked_array(data=[0.01, 0.01, 0.1, 0.1],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_penalty': masked_array(data=['l1', 'l2', 'l1', 'l2'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 0.01, 'penalty': 'l1'},
  {'C': 0.01, 'penalty': 'l2'},
  {'C': 0.1, 'penalty': 'l1'},
  {'C': 0.1, 'penalty': 'l2'}],
 'split0_test_score': array([0.96929685, 0.98234171, 0.98806872, 0.98918231]),
 'split1_test_score': array([0.97056952, 0.98011454, 0.98377346, 0.98552339]),
 'split2_test_score': array([0.97025135, 0.98027362, 0.98520522, 0.98

In [11]:
result_df = pd.DataFrame(grid_test.cv_results_)
result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.643777,0.013854,0.004842,0.001957,0.01,l1,"{'C': 0.01, 'penalty': 'l1'}",0.969297,0.970570,0.970251,0.972792,0.972951,0.971172,0.001450,4
1,1.633351,0.029852,0.003352,0.000486,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",0.982342,0.980115,0.980274,0.982021,0.983612,0.981672,0.001320,3
2,1.938900,0.427301,0.002963,0.000015,0.1,l1,"{'C': 0.1, 'penalty': 'l1'}",0.988069,0.983773,0.985205,0.986317,0.986158,0.985904,0.001411,2
3,2.941890,0.715248,0.003969,0.000018,0.1,l2,"{'C': 0.1, 'penalty': 'l2'}",0.989182,0.985523,0.987910,0.987589,0.988703,0.987782,0.001262,1


In [12]:
y_pred = grid_test.predict(trans_test)
print(metrics.classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6472
           1       0.99      0.99      0.99      6998

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470

